# simple LSTM

so basic it runs on pumpkin spice

## generating data

### echo sequence prediction problem

our echo sequence prediction problem needs data: specifically vectors of random sequences. let's use integers, and define our problem space as integers between 0 and 99.

we'll use the ```randint()``` function from the python 3 ```random``` [module](https://docs.python.org/3/library/random.html "python 3 random module docs") to generate random integers within the range we specify (in this case, 0 to 99). 

we can use the ```randint()``` function within a function of our own to generate sequences of random integers--this will be the data for our problem.

In [27]:
# randint() is inside the python random module

import random

In [28]:
# use randint() to generate a random integer between 0 and 99

rand_int = random.randint(0, 99)

rand_int

71

we need a _lot_ more than one of these. which means it's time to build a function to automate this for us:

In [29]:
def make_seq(seq_length, n_features):
    
    '''
    generate sequences of a given length
    and given number of features
    '''
    return [random.randint(0, n_features - 1) for _ in range(seq_length)]

__demo:__ let's make a sequence with 10 values and 50 features

In [30]:
make_seq(10, 50)

[15, 14, 39, 33, 18, 20, 47, 5, 34, 37]

### one hot encoding

before we can train the model, we have to encode the data into a format that an LSTM can use. the way we encode data matters; choices made here can significantly affect model performance.

to frame this data properly, let's revisit the original problem:

we're trying to predict a number. a _specific_ number.

if we wanted to _approximate_ the number, we could frame this as a __regression__ problem, and train our model to output a close (but not exact) approximation of the number.

but because we want the _exact_ integer (and _not_ an approximation, which is what a regression model outputs) we need to frame this problem as a __classification__ model.

__classification__ means handling categorical data, which machines can do handily using __one hot encoding__.

### automatic vs manual one hot encoding

```scikit-learn``` has a super neat ```OneHotEncoder()``` [transformer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html "sklearn OneHotEncoder doc") that can automate one hot encoding, but because it fits the data, it can only encode the values that it sees represented. 

we need all possible values--from 0 to 99--represented. but because we're generating our integer sequences pseudo-randomly using ```np.random.randint()```, we can't guarantee that all values will be represented.

it's possible to feed in the categories to ```OneHotEncoder()``` manually. here, however, we're going to simply make our own transformer.

we'll convert the results to a ```numpy``` ```array``` in order to make them easier to decode later.

### decoding

later on we'll need a way to interpret the model's results. to do so we'll need to decode the one hot scheme.

we can easily do this using the ```numpy``` ```argmax()``` function.

```numpy.argmax()``` returns the indices for the maximum values along a vector. because each vector in the binary one hot encoding will be a lot of zeroes with a single high value--a ```1```--we can easily use ```argmax()``` to grab the index of the ```1``` value and return it. that's our output.

In [31]:
from numpy import array
from numpy import argmax

# encoder function

def one_hot_encoder(seq, n_features):
    
    '''
    creates a vector of binary values for each
    possible feature in the dataset.
    '''
    
    encoding = list()
    
    for val in seq:
        
        vector = [0 for _ in range(n_features)]
        vector[val] = 1
        encoding.append(vector)
        
    return array(encoding)  

# decoder function

def one_hot_decoder(seq_encoded):
    '''
    decodes results by returning the index of
    the point in the vector with the largest value,
    i.e. 1 
    '''
    
    return [argmax(vector) for vector in seq_encoded]

In [32]:
seq = make_seq(50, 100)

seq_encoded = one_hot_encoder(seq, 100)

seq_decoded = one_hot_decoder(seq_encoded)

print(seq, '\n')
print(seq_encoded, '\n')
print(seq_decoded)

[50, 17, 95, 59, 52, 92, 49, 96, 35, 4, 13, 72, 11, 18, 29, 22, 56, 38, 32, 11, 94, 62, 80, 40, 58, 72, 68, 18, 66, 0, 63, 60, 18, 49, 76, 11, 60, 56, 31, 2, 19, 92, 45, 84, 45, 59, 15, 99, 50, 58] 

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]] 

[50, 17, 95, 59, 52, 92, 49, 96, 35, 4, 13, 72, 11, 18, 29, 22, 56, 38, 32, 11, 94, 62, 80, 40, 58, 72, 68, 18, 66, 0, 63, 60, 18, 49, 76, 11, 60, 56, 31, 2, 19, 92, 45, 84, 45, 59, 15, 99, 50, 58]


### reshape to 3d matrix

LSTMs require input in the form of a 3d matrix.

the three dimensions LSTMs need, in order, are: __samples, time steps, & features__.

the sequence we generated above, ```seq```, is 

* one __sample__,
* fifty __time steps__, 
* one hundred __features__.

for ```seq```, the specific sequence we just generated, it's easy to set the shape to three dimensions using the ```reshape()``` function:

In [33]:
X = seq_encoded.reshape(1, 50, 100)

print(X)
print(X.shape)

[[[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]]
(1, 50, 100)


a more generalizable version might look like this:

```X = seq_encoded.reshape(n_samples, length, n_features)```

### generating samples

following each of the steps above, in order, will generate 1 sample for our LSTM model.

it makes sense to automate these tasks:

In [34]:
def make_sample(length, n_features, output_index):
    '''
    creates a single sample that is LSTM-ready.
    '''
    #create sequence of pseudo-random integers
    seq = make_seq(length, n_features)
    
    # one hot encoding
    seq_encoded = one_hot_encoder(seq, n_features)
    
    # reshape to 3d matrix suitable for LSTM
    X = seq_encoded.reshape(1, length, n_features)
    
    # get the output
    y = seq_encoded[output_index].reshape(1, n_features)
    
    return X, y
    

let's test ```make_sample``` to make sure it works:

In [35]:
X, y = make_sample(50, 100, 17)

print(X, '\n')
print(X.shape, '\n')
print(y, '\n')
print(y.shape)

[[[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]] 

(1, 50, 100) 

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] 

(1, 100)


## building the model

we'll build the model in 3 steps:

1) __define__ and __compile__ the model

2) __fit__ the model

3) __evaluate__ the model

define the model using the ```keras``` API:

In [36]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [37]:
# set the feature number and length in advance for cleaner code

length = 10

n_features = 25

output_index = 7

### create & define the model

this simple model will consist of two layers: an __LSTM__ layer with 25 memory units, and a fully connected __dense__ layer with one neuron per feature.

In [38]:
# initialize a Sequential() model

lstm_model = Sequential()

# add LSTM layer
# 25 memory units

lstm_model.add(LSTM(25, input_shape=(length, n_features)))

# add dense layer
# fully connected = 1 neuron for each feature
# softmax for classification output

lstm_model.add(Dense(n_features, activation='softmax'))


### compile the model

this is where we set parameters like the model's _loss function_, _optimizer_, and the specific performance-related information we want the machine to output as it trains.

this example uses the [log loss function](http://wiki.fast.ai/index.php/Log_Loss "log loss function"), specified under the ```loss``` parameter as ```categorical_crossentropy```.

this model uses the __adam__ optimizer, and outputs accuracy measurements ```acc``` each epoch.

In [39]:
# compile the model

lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', 
                   metrics=['acc'])

### fit the model

using the ```make_sample``` function we created above we could:

1) make a large number of samples

2) put them all together

3) feed them into the model

__however__, let's not do that here. the purpose of this notebook is to explore basic configuration of LSTMs using the ```keras``` api. 

for this model, we'll set one epoch to one sample, and clear the internal state in between epochs. this means our batch size = 1.

this means that the model will train and test on one sample at a time.

setting the ```verbose``` parameter equal to 2 will output training times, loss, and accuracy (either 1 or 0, for 0-100%) for each epoch. 

accuracy will be either 1 or 0 because we're training and testing on one sample at a time.

#### manually fit the model

we will train the model over ten thousand samples/epochs, with a batch size of 1 sample.

it's easy to create a loop that will run the model on a new sample each time, using the ```make_sample``` function from above:

In [40]:
# fit the model

for epoch in range(10000):
    
    X, y = make_sample(length, n_features, output_index)
    
    lstm_model.fit(X, y, epochs=1, verbose=2)

Epoch 1/1
 - 0s - loss: 3.1267 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.3143 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2547 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1354 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 3.3379 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1236 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 3.2076 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1950 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1185 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2956 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2501 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2714 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2436 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2722 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2069 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2336 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1949 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1829 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2219 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2416 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.11

 - 0s - loss: 3.1864 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1843 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2677 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1483 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1198 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2852 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2690 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2983 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.0781 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1760 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1957 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2225 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2342 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2065 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.3303 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1951 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2302 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1994 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1711 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1131 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1498

Epoch 1/1
 - 0s - loss: 3.1215 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1113 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2162 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2600 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1158 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1721 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2710 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.0581 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1553 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.3560 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1987 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1583 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.3290 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2683 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2372 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.0915 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.0467 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 3.2056 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.0440 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 3.2569 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.24

Epoch 1/1
 - 0s - loss: 3.2623 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1226 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1898 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1780 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1357 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.0752 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2292 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2881 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2055 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2817 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.0014 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 3.1397 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2576 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1987 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1317 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2760 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.3334 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1918 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1660 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1673 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 

Epoch 1/1
 - 0s - loss: 3.4249 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.0415 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1569 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2285 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.4659 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.0771 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1207 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.0606 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1768 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1652 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.4496 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.3351 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2831 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.1784 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2989 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.0631 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.2805 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.0823 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 3.1892 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 2.8910 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 3.08

...

Epoch 1/1
 - 0s - loss: 1.0635 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 2.0672 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 0.2982 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.0432 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 2.1118 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.0576 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.6107 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.0078 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.8375 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.0245 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.3439 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.9629 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.9406 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.6874 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2727 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5557 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7921 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.0197 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.7915 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 0.8914 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4657 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.8219 - ac

Epoch 1/1
 - 0s - loss: 1.5560 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.5978 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 2.1465 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 0.3310 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.4885 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 2.9841 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.9291 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.0303 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 2.5642 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.2622 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.7698 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 0.9414 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7519 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.3439 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 0.7273 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3008 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5023 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.9116 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.3749 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.8998 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7704 - acc: 1.0000
Epoch 1/1
 - 0s - l

Epoch 1/1
 - 0s - loss: 0.7750 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.9181 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.6869 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 2.1174 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.3044 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 0.1510 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 2.5089 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 0.9697 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.1488 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 0.9681 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.6182 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5106 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.6562 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.8035 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.5922 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 0.6556 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7021 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4798 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3995 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7194 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.8676 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.2879 - ac

Epoch 1/1
 - 0s - loss: 1.1983 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.5785 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 0.8127 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7664 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.0925 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.1320 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.9877 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3197 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7891 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.4257 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.6699 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.5798 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5554 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5945 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.7425 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 0.5874 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.9064 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.5202 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 0.2240 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.7446 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.6195 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.3

Epoch 1/1
 - 0s - loss: 2.7230 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.3872 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.0999 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 2.3811 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 0.9846 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5742 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7509 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4571 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.4321 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.7599 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.4016 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.0708 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 0.9715 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 2.1090 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.5932 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 0.9772 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.6068 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.9573 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 2.2104 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.4335 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 0.1832 - acc: 1.0000
Epoch 1/1
 - 0s

## more information

##### python 3 random module documentation:

https://docs.python.org/3/library/random.html

##### sklearn preprocessing documentation:

http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing